### Final Activity 2

MegaTon Bank would like to gain insight on their customers. They aim to discover what type of saving plans, money management, loans, and other types of services they can recommend. For this, they collected the usage behavior and debt information of their active credit card holders. The debtData.csv dataset presents the attributes of credit card holders. This dataset can be used for customer segmentation to define a marketing strategy.

Create a clustering model for the given dataset. The dataset has the following columns:
- CustomerID
- Age
- EducationLevel
- YearsEmployed
- Income - in thousands per month
- CardDebt
- OtherDebt
- Defaulted - failed to pay the minimum payment of existing debt
- DebtIncomeRatio

In [1]:
import pandas as pd

raw = pd.read_csv("data/debtData.csv")
raw

,CustomerID,Age,EducationLevel,YearsEmployed,Income,CardDebt,OtherDebt,Defaulted,DebtIncomeRatio
0,1,41,2,6,19,0.124,1.073,0.0,6.3
1,2,47,1,26,100,4.582,8.218,0.0,12.8
2,3,33,2,10,57,6.111,5.802,1.0,20.9
3,4,29,2,4,19,0.681,0.516,0.0,6.3
4,5,47,1,31,253,9.308,8.908,0.0,7.2
...,...,...,...,...,...,...,...,...,...
845,846,27,1,5,26,0.548,1.220,NaN,6.8
846,847,28,2,7,34,0.359,2.021,0.0,7.0
847,848,25,4,0,18,2.802,3.210,1.0,33.4
848,849,32,1,12,28,0.116,0.696,0.0,2.9
